In [11]:
import fitz
from typing import List
from vectordb import Memory

import pymupdf4llm


In [12]:

def formatar(txt):
  txt['text'] = f" $$ pag{str(txt['metadata']['page'])} arq-{txt['metadata']['file_path']} \n\n\n" + txt['text']
  
  return txt



In [13]:
md_text = pymupdf4llm.to_markdown("credenciamento.pdf", page_chunks=True, write_images=False)

md_text = [formatar(i)  for i in md_text]


brute = ''
for i in md_text:
  brute += i['text']

In [14]:
# md_tex_2t = pymupdf4llm.to_markdown("credenciamento.pdf", page_chunks=False, write_images=False)


from langchain.text_splitter import MarkdownHeaderTextSplitter, RecursiveCharacterTextSplitter



padrao = [('$$', 'pagina')]

md_spliter = MarkdownHeaderTextSplitter(headers_to_split_on=padrao)

sections = md_spliter.split_text(brute)




In [15]:
# chunk_size = 20
# chunk_overlap = 3

# text = 'Once you have your data in Markdown format you are ready to chunk/split it and supply it to your LLM, for example, if this is LangChain then do the following:'

# splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)

# chunks = splitter.split_text(text)


In [16]:
memory = Memory(chunking_strategy={"mode": 'sliding_window', "window_size": 500, "overlap": 8})




/home/gabriel/Documentos/estudostccc/env/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [17]:

for i in sections:
  memory.save( i.page_content, i.metadata)


q =  memory.search('Como fazer um requerimento', top_n=10)

q
  


[{'chunk': 'haja interesse da Administração e mantidas as condições e exigências previstas na legislação . 12.1 A inobservância à manutenção das condições e exigências dispostas na parte final do “ caput ” deste artigo caracteriza descumprimento de obrigação e deve ser condenada com a revogação do ato de credenciamento . 12.2 As prorrogações devem ser requeridas pela credenciada , ao menos , 90 ( noventa ) dias antes do término do prazo do credenciamento vigente . 12.3 No ato do requerimento de prorrogação a que se refere o § 2º deste artigo , a entidade interessada em manter o credenciamento deve verificar eventuais alterações ou datas de validade dos documentos anteriormente apresentados e instruir o requerimento com nova documentação para a habilitação , qualificação ou regularidade exigida por este Edital . # # # # # 13 . DISPOSIÇÕES FINAIS 13.1 A SEFAZ / SE pode estabelecer a forma e condições para a fiscalização e prestação de contas das atividades disciplinadas neste Edital . 13

In [18]:
vector = [i['chunk'] for i in memory.search('Como fazer um requerimento', top_n=10)]

t = 'Como fazer um requerimento? use o texto seguinte como base:'
for i in vector:
  t += i

t


'Como fazer um requerimento? use o texto seguinte como base:haja interesse da Administração e mantidas as condições e exigências previstas na legislação . 12.1 A inobservância à manutenção das condições e exigências dispostas na parte final do “ caput ” deste artigo caracteriza descumprimento de obrigação e deve ser condenada com a revogação do ato de credenciamento . 12.2 As prorrogações devem ser requeridas pela credenciada , ao menos , 90 ( noventa ) dias antes do término do prazo do credenciamento vigente . 12.3 No ato do requerimento de prorrogação a que se refere o § 2º deste artigo , a entidade interessada em manter o credenciamento deve verificar eventuais alterações ou datas de validade dos documentos anteriormente apresentados e instruir o requerimento com nova documentação para a habilitação , qualificação ou regularidade exigida por este Edital . # # # # # 13 . DISPOSIÇÕES FINAIS 13.1 A SEFAZ / SE pode estabelecer a forma e condições para a fiscalização e prestação de conta

In [24]:
def searchQuery(question):
  vector = [i['chunk'] for i in memory.search(question, top_n=10)]

  t = f'A pergunta é esta: {question} - a resposta deve ser feita em portugues do brasil e use o texto seguinte como base: '
  for i in vector:
    t += i
  return t


In [20]:
# from openai import OpenAI
# from dotenv import load_dotenv
# from IPython.display import display_markdown


# import os

# load_dotenv()

# ask = "Como fazer um requerimento"

# messages = [{"role": "system", "content": chunk} for chunk in vector]


# client = OpenAI(api_key='ollama', base_url='http://localhost:11435/v1/')

# response = client.chat.completions.create(
#   model='llama3',
#   messages = [
  
#     {"role": "user", "content": t},
#   ]
# )

# print(response.choices[0].message.content)


In [27]:
# from ollama import AsyncClient, Client

# client = AsyncClient(host="http://localhost:11435")

# stream = await client.chat(model='llama3', stream=True, messages = [
  
#     {"role": "user", "content": t},
#   ])

# print(t)
# async for chunck in stream:
#   print(chunck['message']['content'], end='', flush=True)


# t = 3.27


from ollama import Client
import time

client = Client(host='http://localhost:11435')

questions = [
    "Como fazer um requerimento de credenciamento?",
    "Quem supervisiona as operações de crédito e quais as suas demais atribuições?",
    "Qual o procedimento em caso de vazamento de dados pessoais?",
    "Quais equipamentos a parte contratada deve dispor para a realização dos pagamentos?",
    "Uma vez realizado o credenciamento, quais são as funções do prestador de serviço?"
]



def get_response_time(question, client):
    start_time = time.time()
    response = client.chat(model='llama3', messages=[
        {
            'role': 'user',
            'content': question,
        },
    ])
    end_time = time.time()
    elapsed_time = end_time - start_time
    return elapsed_time, response['message']['content']

output_file = 'resultados.txt'

with open(output_file, 'w', encoding='utf-8') as file:
    # Processamento para cada pergunta
    for question in questions:
        file.write('---------------------\n')
        file.write(f"Pergunta: {question}\n\n")

        total_time = 0
        response_times = []

        for i in range(5):
            elapsed_time, response = get_response_time(searchQuery(question), client)
            response_times.append(elapsed_time)
            total_time += elapsed_time
            file.write(f"Resposta {i+1}: {response}\n")
            file.write(f"(Tempo: {elapsed_time:.4f} segundos)\n\n")

        average_time = total_time / 5
        file.write(f"Tempo médio de resposta para a pergunta '{question}': {average_time:.4f} segundos\n")
        file.write(f"Tempos de resposta: {response_times}\n\n")

print(f"Os resultados foram salvos no arquivo '{output_file}'.")



KeyboardInterrupt: 

In [28]:
import pathlib
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [29]:
import os

from dotenv import load_dotenv

load_dotenv()


api_key = os.getenv("GOOGLE_API_KEY")

genai.configure(api_key=api_key)




model = genai.GenerativeModel('gemini-1.0-pro')

response = await model.generate_content_async(t, stream=True)

async for chunk in response:
  print(chunk.to_dict())
  # to_markdown(chunk.text)


{'candidates': [{'content': {'parts': [{'text': '**Modelo de Requerimento para Credenciamento de Entidade para Solução'}], 'role': 'model'}, 'finish_reason': 1, 'index': 0, 'safety_ratings': [{'category': 9, 'probability': 1, 'blocked': False}, {'category': 8, 'probability': 1, 'blocked': False}, {'category': 7, 'probability': 1, 'blocked': False}, {'category': 10, 'probability': 1, 'blocked': False}], 'token_count': 0, 'grounding_attributions': []}], 'usage_metadata': {'prompt_token_count': 4247, 'candidates_token_count': 16, 'total_token_count': 4263}}
{'candidates': [{'content': {'parts': [{'text': ' de Pagamento de Débitos Tributários Estaduais com Uso de Cartão de Crédito junto à SEFAZ/SE**\n\nAo Secretário de Estado da'}], 'role': 'model'}, 'finish_reason': 1, 'index': 0, 'safety_ratings': [{'category': 9, 'probability': 1, 'blocked': False}, {'category': 8, 'probability': 1, 'blocked': False}, {'category': 7, 'probability': 1, 'blocked': False}, {'category': 10, 'probability': 1

- bert eh encoded nao decoded.
- olhar modelos da apple - pegar link com gabes (open elm) https://huggingface.co/apple/OpenELM
- 